In [11]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict


from rl_env import DraftEnv

How do I compute probabilities of what each player will do?
Thinking about what each player needs.

Factors to consider
 - Starters needed - 0 pr weight on people not needed for starters
 - roll out until next turn - for each player in between self and next pick, apply the same rules
 - i.e. for each player, there say there 
 - 

In [98]:

STARTER_COMPOSITION = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DEF": 1, "FLEX": 2}
NUM_DRAFT_ROUNDS = 15
NUM_MGRS = 12

def softmax(x, temperature=1.0):
    x = np.array(x)
    e_x = np.exp((x - np.max(x))/temperature)
    return e_x / e_x.sum(axis=0)

class Players:
    
    def __init__(self):
        self.all_players = self._make_players_df()
        self.open_players = self.all_players.copy()
        self.keepers = {
            0: {"round": 5, "sleeper_id": "8146"},
            1: {"round": 10, "sleeper_id": "2749"},
            2: {"round": 9, "sleeper_id": "9226"},
            3: {"round": 11, "sleeper_id": "8183"},
            4: {"round": 9, "sleeper_id": "1264"},
            5: {"round": 11, "sleeper_id": "5947"},
            6: {"round": 6, "sleeper_id": "8150"},
            7: {"round": 6, "sleeper_id": "10229"},
            8: {"round": 5, "sleeper_id": "6803"},
            9: {"round": 3, "sleeper_id": "2216"},
            10: {"round": 4, "sleeper_id": "5892"},
            11: {"round": 11, "sleeper_id": "10859"}
        }
        keeper_ids = [keeper['sleeper_id'] for keeper in self.keepers.values()]
        self.open_players = self.open_players[~self.open_players['sleeper_id'].isin(keeper_ids)]
    
    
    def _make_players_df(self):
        df_sleeper = pd.read_csv("data/sleeper/all_players.csv")
        df_qb_proj = pd.read_csv("data/projections/QB_projections.csv")
        df_rb_proj = pd.read_csv("data/projections/RB_projections.csv")
        df_wr_proj = pd.read_csv("data/projections/WR_projections.csv")
        df_te_proj = pd.read_csv("data/projections/TE_projections.csv")
        df_k_proj = pd.read_csv("data/projections/K_projections.csv")
        df_def_proj = pd.read_csv("data/projections/DEF_projections.csv")



        df_qb_proj = df_qb_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_rb_proj = df_rb_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_wr_proj = df_wr_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_te_proj = df_te_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_k_proj = df_k_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_def_proj = df_def_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)

        df_proj = pd.concat([df_qb_proj, df_rb_proj, df_wr_proj, df_te_proj, df_k_proj, df_def_proj])

        df_proj_agg = df_proj.groupby('sleeper_id')['fpts'].agg(['mean', 'std']).reset_index()
        df_proj_agg['sleeper_id'] = df_proj_agg['sleeper_id'].astype(str)


        df_players = df_proj_agg.merge(df_sleeper.loc[:, ['sleeper_id', 'full_name', 'position', 'team']], 
                                        on='sleeper_id', 
                                        how='left')
        
        df_players = df_players.dropna()
        return df_players


    
    

In [108]:
class Manager:
    def __init__(self, mgr_num: int, players: Players):
        
        self.mgr_num = mgr_num
        self.team = pd.DataFrame({
            "round": list(range(15)),
            "sleeper_id": None,
            "full_name": None,
            "position": None,
            "team_pos": None,
            "team": None,
            "fp_mean": None,
            "fp_std": None
        })
        self.players = players
        self.open_players = self.players.open_players
        self.all_players = self.players.all_players
        
    def flex_ready(self) -> bool:
        ''' Check if the manager has filled their flex positions '''
        team_comp = self.get_team_comp(flex=False)
        flex_ready = team_comp.get("RB") >= STARTER_COMPOSITION["RB"] and \
            team_comp.get("WR") >= STARTER_COMPOSITION["WR"] and \
            team_comp.get("TE") >= STARTER_COMPOSITION["TE"]
        return flex_ready
        
    def _assign_player(self, round: int, **kwargs):
        for key, value in kwargs.items():
            self.team.loc[self.team["round"] == round, key] = value
    
    def add_player(self, sleeper_id: str, round: int):
        player = self.open_players[self.open_players["sleeper_id"] == sleeper_id]
        assert len(player) == 1, f"Player {sleeper_id} not found"
        player = player.iloc[0]
        if self.flex_ready() and player['position'] in ["RB", "WR", "TE"]:
            team_pos = "FLEX"
        else:
            team_pos = player['position'] 
        self._assign_player(round, 
                           sleeper_id=player["sleeper_id"], 
                           full_name=player["full_name"], 
                           position=player["position"], 
                           team_pos=team_pos, 
                           team=player["team"], 
                           fp_mean=player["mean"], 
                           fp_std=player["std"])
        
        self.open_players = self.open_players[self.open_players["sleeper_id"] != sleeper_id]
        

    
    def get_team_comp(self, flex=True) -> Dict[str, int]:
        ''' Get the team composition for a manager.
        If flex is True, then the FLEX position is included in the count'''
        if flex:
            team_comp = self.team.groupby("team_pos").size().to_dict()
            team_comp["FLEX"] = team_comp.get("FLEX", 0)
        else:
            team_comp = self.team.groupby("position").size().to_dict()
        for pos in ["QB", "RB", "WR", "TE", "K", "DEF"]:
            # fill in missing positions with 0
            team_comp[pos] = team_comp.get(pos, 0)
        return team_comp
    
    def get_needed_pos_counts(self, flex=True) -> Dict[str, int]:
        ''' Get the number of each position needed for a manager to fill their starters '''
        team_comp = self.get_team_comp(flex=flex)
        needed_pos_counts = {pos: STARTER_COMPOSITION[pos] - team_comp[pos] for pos in STARTER_COMPOSITION}
        return needed_pos_counts
    
    def stochastic_choice(self, temperature=1) -> dict:
        '''
        picks a random (intelligent) player for the NPC managers.
        unlike reasonable_action methods, this returns the player as a dict
        This allows us to choose players who are not the highest ranked player in a position
        '''
        
        team_comp = self.get_team_comp(flex=True)
        needed_pos_counts = self.get_needed_pos_counts(flex=True)
        
        # first get what non-flex positions are needed
        needed_positions = [pos for pos, count in needed_pos_counts.items() if pos != "FLEX" and count > 0]
        rb_te_wr_filled = team_comp.get("RB", 0) >= STARTER_COMPOSITION["RB"] and \
            team_comp.get("WR", 0) >= STARTER_COMPOSITION["WR"] and \
            team_comp.get("TE", 0) >= STARTER_COMPOSITION["TE"]
        
        # if the rb, wr, and te are filled, check flex
        if needed_pos_counts.get("FLEX", 0) > 0 and rb_te_wr_filled:
            # don't worry about flex if rb, wr, te are not filled yet
                needed_positions += ["RB", "WR", "TE"]
                
        # if we need players, choose the highest point player available of any needed position
        if len(needed_positions) != 0:
            needed_players = self.open_players[self.open_players['position'].isin(needed_positions)]
            if not needed_players.empty:
                # if players are available in a needed position, choose the highest point player in a needed position
                # select up to 5 of the top players, with a max of 100 mean less than the top
                options = needed_players.loc[needed_players["mean"] >= needed_players["mean"].max() - 100]
                options = options.iloc[:np.min([5, len(options)])]
                w = softmax(options["mean"].values, temperature=temperature)
                chosen_player = options.sample(1, weights=w).iloc[0]
                
            else:
                # if no players are available in a needed position, choose the highest point player available
                # chosen_player = self.open_players.iloc[0]
                options = self.open_players.loc[self.open_players["mean"] >= self.open_players["mean"].max() - 100]
                options = options.iloc[:np.min([5, len(options)])]
                w = softmax(options["mean"].values, temperature=temperature)
                chosen_player = options.sample(1, weights=w).iloc[0]
                
        else: 
            # if we don't need players, choose the highest point player available
            # chosen_player = self.open_players.iloc[0]
            options = self.open_players.loc[self.open_players["mean"] >= self.open_players["mean"].max() - 100]
            options = options.iloc[:np.min([5, len(options)])]
            w = softmax(options["mean"].values, temperature=temperature)
            chosen_player = options.sample(1, weights=w).iloc[0]
            
        # print(w)
        return chosen_player.to_dict()

players = Players()
ashish = Manager(1, players)
justin = Manager(0, players)
choice = ashish.stochastic_choice()
print(choice)
ashish.add_player(choice['sleeper_id'], round=1)

justin.players.open_players.loc[justin.players.open_players["position"] == "RB"].sort_values(by="mean", ascending=False).head(10)
# ashish.open_players.loc[justin.open_players["position"] == "RB"].sort_values(by="mean", ascending=False).head(10)


{'sleeper_id': '4034', 'mean': 335.53424699283335, 'std': 25.370946800430414, 'full_name': 'Christian McCaffrey', 'position': 'RB', 'team': 'SF'}


,sleeper_id,mean,std,full_name,position,team
103,4034,335.534247,25.370947,Christian McCaffrey,RB,SF
487,8155,277.845540,25.676804,Breece Hall,RB,NYJ
600,9509,270.806667,19.260665,Bijan Robinson,RB,ATL
154,4866,260.333579,22.280809,Saquon Barkley,RB,PHI
300,6813,252.155880,13.585306,Jonathan Taylor,RB,IND
568,9221,239.975561,24.997244,Jahmyr Gibbs,RB,DET
384,7543,232.434841,22.864002,Travis Etienne,RB,JAX
513,8205,227.511870,19.167158,Isiah Pacheco,RB,KC
217,5850,225.871830,26.584809,Josh Jacobs,RB,GB
471,8136,225.523241,20.575745,Rachaad White,RB,TB


In [ ]:
import pymc as pm
import numpy as np


# Instantiate managers
managers = [Manager(i, players) for i in range(NUM_MGRS)]

# Run the simulation
with pm.Model() as model:
    for round_num in range(NUM_DRAFT_ROUNDS):
        print(f"Round {round_num + 1}")
        for manager in managers:
            pick = manager.stochastic_choice()
            trace = pm.sample_prior_predictive(samples=1)
            
            # Accessing the prior group
            prior_samples = trace.prior
            
            # Retrieve the sampled index and ensure it's an integer
            picked_player_idx = int(prior_samples[f'{manager.name}_pick'].values[0])
            picked_player = manager.players[picked_player_idx]
            print(f"{manager.name} picked {picked_player}")
            manager.pick_history.append(picked_player)
            
            # Remove the picked player from all managers' player pools
            for m in managers:
                m.players = np.delete(m.players, picked_player_idx)
                m.positions = np.delete(m.positions, picked_player_idx)
                m.player_scores = np.delete(m.player_scores, picked_player_idx)

        print()

# Review the picks made by each manager
for manager in managers:
    print(f"{manager.name}'s picks: {manager.pick_history}")


In [90]:
players = Players()
ashish = Manager(1, players)
ashish.stochastic_choice()


[5.86889370e-20 5.82638582e-15 1.87497321e-02 7.34786457e-20
 9.81250268e-01]


{'sleeper_id': '4034',
 'mean': 335.53424699283335,
 'std': 25.370946800430414,
 'full_name': 'Christian McCaffrey',
 'position': 'RB',
 'team': 'SF'}